## Convert dataset to pytorch data

In [1]:

import os
import os.path as osp
import torch

from torch_geometric.data import DataLoader, Dataset

import numpy as np
import torch.nn.functional as F




class ChallengeDataset(Dataset):
    """
    Base class representing a dataset for the challenge.
    
    We assume that the conversion process is already done, i.e. we 
    work with a list of pytorch Data objects stored in .pt files,
    all in the same folder.
    """
    

    def challenge_transform(self,data,converted_path=None,debug=False):
        all_timeparams = ['EqLambda', 'AvgPktsLambda', 'ExpMaxFactor',
                     'MinPktLambda','MaxPktLambda','StdDev','PktsLambdaOn',
                      'AvgTOff','AvgTOn','BurstGenLambda','Bitrate',
                      'ParetoMinSize','ParetoMaxSize','ParetoAlfa'
                     ]

        all_sizeparams = ['MinSize','MaxSize','AvgPktSize','PktSize1',
                         'PktSize2','NumCandidates','Size_i','Prob_i']


        """ 1a. Assert that some stuff remains constant..."""
        if debug:
            assert all(data.p_SizeDist == 2)
            assert all(data.p_TimeDist == 0)
            assert all(data.n_levelsQoS == 1)
            assert all(data.p_ToS == 0.0)
            assert all(data.n_queueSizes == 32)
            assert data.n_schedulingPolicy[0] == "FIFO"
            for a,v in zip(['p_size_AvgPktSize','p_size_PktSize1',
                            'p_size_PktSize2', 'p_time_ExpMaxFactor'],
                        [1000.0,300.0,1700.0,10.0]):
                if not torch.allclose(getattr(data,a), 
                                      v*torch.ones_like(getattr(data,a)),rtol=1e-05):
                    raise Exception(f"{a} was expected to have the value close to {v}")

        del data.p_SizeDist, data.p_TimeDist, data.p_ToS
        del data.n_queueSizes, data.n_levelsQoS, data.n_schedulingPolicy

        """ 1b. Transform p_SizeDist and p_TimeDist into one-hot. We skip it because it 
        does not change in the training dataset.
        """
        #data.p_SizeDist= (F.one_hot(data.p_SizeDist,4))
        #data.p_TimeDist= (F.one_hot(data.p_SizeDist,6))

        """ 
        2. Path attributes; Concatenate Size/Time distribution parameters.

         It turns out all sizeparams have the same value. Useless.
            Otherwise, we'd have:
                data.p_sizeparams = torch.cat([getattr(data,a).view(-1,1) for a in sizeparams],axis=1)

         Also, p_time_ExpMaxFactor is always equal to 10.0, so we delete it
        """
        delattr(data,'p_time_ExpMaxFactor')

        sizeparams = [f'p_size_{a}' for a in ['AvgPktSize','PktSize1',
                     'PktSize2']]
        timeparams = [f'p_time_{a}' for a in ['EqLambda', 'AvgPktsLambda']]

        p_params = timeparams + ['p_TotalPktsGen','p_PktsGen','p_AvgBw']
        data.P = torch.cat([getattr(data,a).view(-1,1) for a in p_params],axis=1)
        for p in p_params + sizeparams:
            delattr(data,p)

        """
            3. Global Attributes
        """
        global_attrs = ['g_delay','g_packets','g_losses','g_AvgPktsLambda']
        data.G = torch.as_tensor([getattr(data,a) for a in global_attrs],device=data.P.device)
        data.G = torch.tile(data.G.view(1,-1),(data.type.shape[0],1))
        for a in global_attrs:
            delattr(data,a)

        """
            4. Link attributes
        """
        data.L = data.l_capacity.clone().view(-1,1)
        delattr(data,'l_capacity')
        
        
        if not converted_path is None:
            torch.save(data,converted_path)

        return data
    
    def __init__(self, root_dir,filenames=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the .pt files.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        if filenames is None:
            onlyfiles = [f for f in os.listdir(self.root_dir) if osp.isfile(osp.join(self.root_dir, f))]
            self.filenames = [f for f in onlyfiles if f.endswith('.pt')]
        else:
            self.filenames = filenames
    

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        filename = self.filenames[idx]
        pt_path = osp.join(self.root_dir, filename) 
        
        converted_dir = self.root_dir+'_2'
        os.makedirs(converted_dir,exist_ok=True)
        converted_path = osp.join(converted_dir, filename) 
        
        sample = torch.load(pt_path,map_location='cuda')

        sample = self.challenge_transform(sample,converted_path=None)

        return sample

    

In [2]:
from tqdm import tqdm,trange

BATCH_SIZE = {'train':1,'val':1}
datasets = {"train": ChallengeDataset(root_dir='/tmp/converted_train'),
            "val":ChallengeDataset(root_dir='./dataset/converted_validation')
           }

dataloaders = {}
for k in datasets.keys():
    if k.startswith('train'):
        batch_size = BATCH_SIZE['train']
    else:
        batch_size = BATCH_SIZE['val']
    dataloaders[k] = DataLoader(datasets[k],batch_size=batch_size,shuffle=True)


for mode in datasets.keys():
    print(f"Converting dataset ({mode})")
    for i,sample in tqdm(enumerate(dataloaders[mode]),
                             total=len(dataloaders[mode])):
        pass
    


  0%|          | 0/120000 [00:00<?, ?it/s]

Converting dataset (train)


  7%|▋         | 8601/120000 [00:42<09:13, 201.25it/s]


KeyboardInterrupt: 


## Divide validation datasets into 3. Initialize datasets/dataloaders


In [2]:
ds_val =  ChallengeDataset(root_dir='./dataset/converted_validation')
filenames = ds_val.filenames

import re
import pandas as pd
import numpy as np
import datanetAPI
import os.path as osp
def converted_filenames_metadata(filenames,path_to_original_dataset):
        import re
        
        def m(f):
            g = re.match("(validation|train)\_(\d+)\_(\d+).*",f).groups()
            g  = [g[0], int(g[1]), int(g[2])]
            return g
        
        matches =  [m(f) for f in filenames]
        reader = datanetAPI.DatanetAPI(path_to_original_dataset)
        files_num = np.array([m[1] for m in matches],dtype=np.int32)
        all_paths = np.array(reader.get_available_files())
        print(all_paths[0,1])
        df = pd.DataFrame(index=filenames,columns=['full_path','num_nodes','validation_setting'])
        df['full_path'] = all_paths[files_num,0]
        df['num_nodes'] = np.array([osp.split(f)[-1] for f in df['full_path'].values],dtype=np.int32)
        
        if matches[0][0] == 'validation':
            df['validation_setting'] = np.array([osp.split(f)[-2][-1] for f in df['full_path'].values],dtype=np.int32)
        else:
            df['validation_setting'] = -1
        return df
        
        
df_val = converted_filenames_metadata(ds_val.filenames,f'./dataset/gnnet-ch21-dataset-{"validation"}')
"""
    We opt to make the validation set smaller (as it is more time consuming to run)
"""
print(df_val['validation_setting'].value_counts())
df_val['filenames'] = df_val.index.values
df_val = df_val.groupby('full_path').head(3)


results_100_400-2000_0_9.tar.gz
3    1040
1    1040
2    1040
Name: validation_setting, dtype: int64


In [3]:
BATCH_SIZE = {'train':32,'val':8}
datasets = {"train": ChallengeDataset(root_dir='/tmp/converted_train'),
            "val":ChallengeDataset(root_dir='./dataset/converted_validation')
           }
for i in range(3):
    which_files = list(df_val[df_val['validation_setting']==i+1]['filenames'].values)
    ds = ChallengeDataset(root_dir='./dataset/converted_validation',
                         filenames=which_files)
    datasets[f'val_{i+1}'] = ds
    
    
    
    
dataloaders = {}
for k in datasets.keys():
    if k.startswith('train'):
        batch_size = BATCH_SIZE['train']
    else:
        batch_size = BATCH_SIZE['val']
    dataloaders[k] = DataLoader(datasets[k],batch_size=batch_size,shuffle=False)
    dataloaders[k+"_s"] = DataLoader(datasets[k],batch_size=batch_size,shuffle=True)
    print(f"Dataset Len ({k}): {len(datasets[k])}")
    print(f"Dataloader Len ({k}): {len(dataloaders[k])}")



Dataset Len (train): 120000
Dataloader Len (train): 3750
Dataset Len (val): 3120
Dataloader Len (val): 390
Dataset Len (val_1): 78
Dataloader Len (val_1): 10
Dataset Len (val_2): 78
Dataloader Len (val_2): 10
Dataset Len (val_3): 78
Dataloader Len (val_3): 10


## Calculate Mean, Std Deviation of training dataset

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats._continuous_distns import _distn_names
import matplotlib

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=100, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Best holders
    best_distribution = None
    lowest_error = 9999.9

    # Estimate distribution parameters from data
    for ii, distribution in enumerate([d for d in _distn_names if not d in ['levy_stable', 'studentized_range']]):

        print("{:>3} / {:<3}: {}".format( ii+1, len(_distn_names), distribution ))

        distribution = getattr(st, distribution)

        # Try to fit the distribution

        # Ignore warnings from data that can't be fit
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            fail = False
            try:
                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))
                # if axis pass in add to plot
            except:
                fail = True
            if not fail:
                print(sse)
                if sse < 15:
                    fig, ax = plt.subplots()
                    ax.hist(data,bins=bins,color='white',edgecolor='black',density=True)
                    ax.plot(x,pdf)
                    plt.show()

                if sse < lowest_error:
                    lowest_error = sse
                    print(f"New best distribution sse: {sse}")
                    # identify if this distribution is better
                    best_distribution = (distribution, params, sse)
                    
                    



    
    return best_distribution

def get_train_distribution_statistics(ds_train,attrs_to_normalize,calc_distr=False):
    torch.manual_seed(42)
    dl_train_oneshot = DataLoader(ds_train,batch_size=500,shuffle=True)
    for sample in dl_train_oneshot:
        print(sample)
        means=dict([(k, torch.mean(getattr(sample,k).float(),axis=0))\
                   for k in attrs_to_normalize])
        std=dict([(k, torch.std(getattr(sample,k).float(),axis=0))\
                   for k in attrs_to_normalize])
        sample.out_occupancy.cpu().numpy()
        distr = None
        if calc_distr:
            distr =  (best_fit_distribution(sample.out_occupancy.cpu().numpy()))
            plt.show()
            plt.hist(sample.out_occupancy.cpu().numpy()*100.0,bins=100)
            plt.show()
        break
    return means, std, distr

means, stds, _ = get_train_distribution_statistics(datasets['train'],['P','G','L','out_occupancy','out_delay'])



Batch(G=[817852, 4], L=[90852, 1], P=[708410, 5], batch=[817852], edge_index=[2, 3322522], edge_pos=[3322522], edge_pos_rel=[3322522], edge_type=[3322522], out_delay=[708410], out_occupancy=[90852], ptr=[501], type=[817852])


In [37]:
import torch
import pandas as pd
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn.conv import GATConv,TransformerConv,GCNConv

from torch_geometric_temporal.nn.recurrent import GConvGRU
import torch.nn.functional as F
from torch_scatter import scatter
class ChallengeModel(torch.nn.Module):

    
    def __init__(self,num_iterations=3,G_dim = 4,P_dim =5,L_dim = 1,**kwargs):
        super(ChallengeModel, self).__init__(**kwargs)
        self.num_iterations = num_iterations
        self.G_dim = G_dim
        self.P_dim = P_dim
        self.L_dim = L_dim
        
        self.H = 64
        self.H_p = 64
        self.H_l = 64
        self.H_n = 64
        
        self.conv_pn_1 = []
        self.conv_ln_1 = []
        self.conv_pn_r_1 = []
        self.conv_ln_r_1 = []
        self.conv_pl_1 = []
        self.conv_pl_r_1 = []
        
        for i in range(self.num_iterations):
            self.conv_pn_1.append(GCNConv(self.H_p+self.P_dim,self.H_n).cuda())
            self.conv_ln_1.append(GCNConv(self.H_l+L_dim,self.H_n).cuda())
            self.conv_pl_1.append(GCNConv(self.H_p+P_dim,self.H_l).cuda())

            self.conv_pn_r_1.append(GCNConv(self.H_n+G_dim,self.H_p,flow="target_to_source").cuda())
            self.conv_ln_r_1 .append(GCNConv(self.H_n+G_dim,self.H_l,flow="target_to_source").cuda())
        
            self.conv_pl_r_1.append(GConvGRU(self.H_l+L_dim,self.H_p,K=2).cuda())
        
        self.finalconv = GCNConv(self.H_p+P_dim,self.H_l,normalize=True).cuda()
        
        self.lin1 = torch.nn.Linear(L_dim+self.H_l,512)
        self.lin2 = torch.nn.Linear(512,512)
        self.lin3 = torch.nn.Linear(512,1)
        self.xlin1 = torch.nn.Linear(self.H_n+self.G_dim+self.H_p+self.P_dim+self.H_l+self.L_dim,128)
        self.xlin2 = torch.nn.Linear(128,self.H_n+self.G_dim+self.H_p+self.P_dim+self.H_l+self.L_dim)
        
    
    def forward(self,data,means,stds,naive=True):
        edge_index = data.edge_index.long()
        edge_type = data.edge_type
        edge_pos_rel = data.edge_pos_rel.clone()
        edge_pos = data.edge_pos.clone()
        
        is_p = data.type==0
        is_l = data.type==1
        is_n = data.type==2       
        
        
        all_edges= [[] for _ in range(3)]
        for et in [0,1,2]:
            et_pos = data.edge_pos[edge_type==et]
            et_edges = edge_index[:,edge_type==et]
            
            init_tensor = torch.cat([torch.ones(1,device=et_edges.device).long(),torch.diff(et_edges[0,:])],dim=0)
            init_tensor = torch.clip(torch.abs(init_tensor),0.,1.)
            # [1, 0, 0, 0, 1, 0, 0, 1] where 0 iff edge source equal
            """  Debug: 
            init_tensor = torch.as_tensor([1,0, 0, 0, 1, 0, 0, 1] ) 
            sol =  [0,1,2,3,0,1,2,0]
            """
            # [0, 1, 1, 1, 0, 1, 1, 0] where 1 iff edge source equal
            init_tensor = 1 - init_tensor
            # [0, 1, 1, 1, -4, 1, 1, -3] where 1 iff edge source equal
            count_tensor = torch.nonzero(1-init_tensor).view(-1)
            init_tensor[count_tensor[1:]] = -torch.diff(count_tensor) +1
            # [0, 1, 2, 3, 0, 1, 2, 0] where 1 iff edge source equal
            init_tensor = init_tensor.cumsum(axis=0)
            
            # Will list all dsts that were the first linked to some src, then all second, etc..
            ensure_stable = torch.linspace(start=0.0,end=0.5,steps=init_tensor.shape[0],device=init_tensor.device)
            encountered_order = torch.sort(init_tensor+ensure_stable)[1]
            et_edges = et_edges[:,encountered_order]
            
            idxs, vals = torch.unique(init_tensor,return_counts=True)
            vs = torch.split_with_sizes(et_edges,tuple(vals),dim=1)
            all_edges[et] = vs
            #for i,el in enumerate(vs):
            #    print(f"Size of #encountered_at_{i} : ({vs[i].shape})")

        
        edges_pl = edge_index[:,edge_type==0]
        edges_pn = edge_index[:,edge_type==1]
        edges_ln = edge_index[:,edge_type==2]    
        
        #print(f"Edges between path and link: {edges_pl.shape[1]}")
        #print(f"Edges between path and node: {edges_pn.shape[1]}")
        #print(f"Edges between link and node: {edges_ln.shape[1]}")
    
        
        G_dim, P_dim, L_dim = self.G_dim, self.P_dim, self.L_dim
        H_n, H_p, H_l = self.H_n, self.H_p, self.H_l
        
        n_p = torch.sum(is_p)
        n_l = torch.sum(is_l)
        n_n = torch.sum(is_n)
        
        P = (data.P - means['P'])/stds['P']
        G = (data.G - means['G'])/stds['G']
        #G[:,0] = data.G[:,0]
        L = (data.L - means['L'])/stds['L']
        #print(f"Paths ({n_p}) Links ({n_l}) Nodes({n_n})")
        
        
        X = torch.zeros(data.G.size(0),H_n+G_dim+H_p+P_dim+H_l+L_dim,device='cuda')
        cnt = 0
        cnt, node_hidden = cnt+H_n,   slice(cnt,cnt+H_n)
        cnt, node_og     = cnt+G_dim, slice(cnt,cnt+G_dim)
        cnt, link_hidden = cnt+H_l,   slice(cnt,cnt+H_l)
        cnt, link_og     = cnt+L_dim, slice(cnt,cnt+L_dim)
        cnt, path_hidden = cnt+H_p,   slice(cnt,cnt+H_p)
        cnt, path_og     = cnt+P_dim, slice(cnt,cnt+P_dim)
        
        cnt = 0
        cnt, node_all = cnt+H_n+G_dim, slice(cnt,cnt+H_n+G_dim)
        cnt, link_all = cnt+H_l+L_dim, slice(cnt,cnt+H_l+L_dim)
        cnt, path_all = cnt+H_p+P_dim, slice(cnt,cnt+H_p+P_dim)
        
        
        X[:,node_og] = G[:,:]
        #print(X)
        #print(L)
        X[is_l,link_og] = L
        X[is_p,path_og] = P
        
        X = F.leaky_relu(self.xlin1(X))
        X = F.leaky_relu(self.xlin2(X))
        
        #print(X[is_n,node_hidden].shape)
        #print(X[is_l,link_hidden].shape)
        #print(X[is_p,path_hidden].shape)
        def act(x):
            return F.leaky_relu(x)
        for i in range(self.num_iterations):
            X[is_p,path_hidden] =  act(self.conv_pn_r_1[i](X[:,node_all].clone(),edges_pn)[is_p,:])
            
            x = X[:,link_all].clone()
            H = None
            max_numpaths = int(vs[0].shape[0])
            for k in range(max_numpaths):
                e = torch.cat([vs[k][1,:].unsqueeze(0),vs[k][0,:].unsqueeze(0)],axis=0)
                H  = self.conv_pl_r_1[i](X=x,H=H,edge_index=e)    
            X[is_p,path_hidden] = act(H[is_p,:]/max_numpaths)
            
            X[is_n,node_hidden] = \
                act(self.conv_pn_1[i](X[:,path_all].clone(), edges_pn)[is_n,:] +\
                              self.conv_ln_1[i](X[:,link_all].clone(),edges_ln)[is_n,:])

            
            X[is_l,link_hidden] =  act(self.conv_ln_r_1[i](X[:,node_all].clone(),edges_ln)[is_l,:])
            X[is_l,link_hidden] =  act(self.conv_pl_1[i](X[:,path_all].clone(),
                                                         vs[k])[is_l,:])
            
        
        #X[is_l,link_hidden] =  self.finalconv(X[:,path_all].clone(),edges_pl)[is_l,:] 
            
        
        L = X[is_l,link_all]
        L = self.lin1(L)
        #L = F.leaky_relu(L)
        #L = self.lin2(L)
        L = F.leaky_relu(L)
        L = torch.sigmoid(self.lin3(L)) 
        lamb = (1/0.05)
        L = -(1/lamb)* torch.log(1-0.99*L)
        #L = torch.clip(L,0.0,0.05)
        #L = torch.ones_like(L)*0.044 + 1e-08*L
        """
            When traffic doubles, w, uhhh, expect the avg utilization to double?
        """
        #L = L * (data.G[:,0]/means['G'][0])
        
        #print(f"Final L shape:{L.shape}")
        
        # Set row nodes to be equal to N
        
        X = torch.zeros(X.size(0), device=X.device)
        
        
        """ To predict the node, we use the formula:
        
            path delay ~= \sum_{i=0}^{n_links} delay_link(i)
            where 
                delay_link(i) := avg_utilization_{i} * (queue_size_{i}/link_capacity_{i})
                
            Our NN predicts avg_utilization_{i} \in [0,1], \forall i. 
            For this dataset, we have \forall i: queue_size_{i} =32000
        """
        X[is_l] = L.squeeze(-1)  * 32000.0  / data.L.squeeze(-1) 
        E   = torch.gather(X,index=edges_pl[1,:],dim=0)
        
        #print(f"Shape after gather {E.shape}")
        res = scatter(src=E,index=edges_pl[0,:],dim=0,dim_size=X.size(0),reduce='sum')
        res = res[is_p]
        
        #print(f"Final per-path-mean delay shape:{res.shape}")
        #if naive:
        #    return sample.G[sample.batch[is_p],0] + 0*res
        return res, L
        
        
        
    
    

In [44]:
from tqdm import tqdm,trange
from convertDataset import total_samples
from torch.nn import MSELoss
def MAPE(preds,actuals):
    return 100.0*torch.mean(torch.abs((preds-actuals)/actuals))
def lMAPE(preds,actuals):
    return 100.0*torch.mean(torch.abs((torch.log(preds)-torch.log(actuals))/actuals))

def MSE(preds,actuals):
    return torch.sqrt(torch.mean(torch.square(preds-actuals)))
import numpy as np

model = ChallengeModel().cuda()

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
num_epochs = 10000
opt = torch.optim.Adam(lr=1e-03,params=model.parameters())
step = 0
#model.load_state_dict('./model/model_3650.pt')
torch.manual_seed(42)
        
    
        
for epoch in range(num_epochs):
    
        
    for mode in ['train','val_1','val_2','val_3']:
        if mode == 'train':
                model.train()
        else:
                model.eval()
        stats = {'loss':[],
                 'out':[],
                 'occup':[],
                 'actual_out':[],
                 'actual_occup':[]
                }
                
        running_loss = []
        out_mean = []
        occup_mean = []
        actual_out_mean = []
    
        cnt = 0
        total=len(dataloaders[mode])//10 if mode == 'train' else len(dataloaders[mode])
        mode_maybe_shuffle = 'train_s' if mode == 'train' else mode
        for i,sample in tqdm(enumerate(dataloaders[mode_maybe_shuffle]),
                             total=total):
            
            with torch.set_grad_enabled(mode == 'train'):
                if i == total-1:
                    break
                if mode == 'train':
                    opt.zero_grad()
                cnt += 1
                out, occup = model(sample,means,stds)

                loss = MAPE(out,sample.out_delay)
                if mode == 'train':
                    MSE(occup,sample.out_occupancy).backward()
                    #(loss).backward()
                    opt.step()
                elif i == 0:
                    print(torch.cat([out.view(-1,1),sample.out_delay.view(-1,1)],axis=1))


                _stats = {'loss':loss,
                          'out':out.mean(),
                          'actual_out':sample.out_delay.mean() if not mode == 'test' else -1.0,
                          'occup':occup.mean(),
                          'actual_occup':sample.out_occupancy.mean() if not mode == 'test' else -1.0,
                         }
                for k in _stats.keys():
                    stats[k].append(_stats[k].cpu().item())
                del _stats
        
        print("======================================")
        print(f"Epoch {epoch} - Avg stats ({mode})")
        for k in stats.keys():
            print(f'Mean {k}: {np.array(stats[k]).mean()}')
            writer.add_scalar(f"{k}/{mode}", np.array(stats[k]).mean(), step)
    writer.flush()
    print("Flushed")
    step += 1

    torch.cuda.empty_cache()

    import os
    os.makedirs('./model',exist_ok=True)
    if i%1 == 0:
        torch.save(model.state_dict(),f'./model/model_{i}.pt')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0 - Avg stats (train)
Mean loss: 31.144194363272764
Mean out: 0.11991727656938812
Mean occup: 0.05404121003407527
Mean actual_out: 0.14383227553437739
Mean actual_occup: 0.05523590333162145


 10%|█         | 1/10 [00:00<00:02,  3.85it/s]

tensor([[0.4719, 0.3901],
        [0.5079, 2.9697],
        [0.5521, 2.1521],
        ...,
        [0.3453, 0.2840],
        [0.3201, 0.2532],
        [0.3368, 0.9066]], device='cuda:0')


  0%|          | 0/10 [00:00<?, ?it/s]

...

tensor([[0.0453, 0.0299],
        [0.0732, 0.0533],
        [0.0837, 0.0577],
        ...,
        [0.0421, 0.0300],
        [0.0421, 0.0289],
        [0.0506, 0.0345]], device='cuda:0')


  0%|          | 0/375 [00:00<?, ?it/s]

Epoch 92 - Avg stats (val_3)
Mean loss: 45.39728927612305
Mean out: 0.02078152261674404
Mean occup: 0.04176405113604334
Mean actual_out: 0.03577864170074463
Mean actual_occup: 0.06337069720029831
Flushed


 72%|███████▏  | 270/375 [01:15<00:29,  3.57it/s]


KeyboardInterrupt: 

In [ ]:
for k in stats.keys():
    print(f'Mean {k}: {np.array(stats[k]).mean()}')
    writer.add_scalar(f"{k}/{mode}", np.array(stats[k]).mean(), step)
step += 1

torch.cuda.empty_cache()
writer.flush()

[('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_0_24.tar.gz')][('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_1000_1024.tar.gz')][('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_10000_10024.tar.gz')]


[('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_10050_10074.tar.gz')]
[('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_10025_10049.tar.gz')]

0it [00:00, ?it/s]

25it [00:02, 11.40it/s]


[('./dataset/gnnet-ch21-dataset-train/30', 'results_30_400-2000_10075_10099.tar.gz')]


25it [00:02, 11.24it/s]


...

25it [00:06,  4.02it/s]


[('./dataset/gnnet-ch21-dataset-train/50', 'results_50_400-2000_9950_9974.tar.gz')]


25it [00:06,  4.03it/s]


[('./dataset/gnnet-ch21-dataset-train/50', 'results_50_400-2000_9975_9999.tar.gz')]


25it [00:06,  4.03it/s]
25it [00:05,  4.29it/s]
25it [00:05,  4.46it/s]
25it [00:05,  4.62it/s]
25it [00:05,  4.60it/s]
